# IMDB 영화 리뷰 감정 분석 (임베딩 레이어 + 양방향 LSTM)

이 노트북은 IMDB 영화 리뷰 데이터셋을 활용하여 감정 분석을 수행합니다.

## 주요 특징
- **데이터**: IMDB 영화 리뷰 (긍정/부정 이진 분류)
- **전처리**: 텍스트 벡터화 (정수 시퀀스)
- **임베딩**: Keras Embedding 레이어 사용
- **모델**: 양방향 LSTM (Bidirectional LSTM)
- **평가**: 훈련/검증/테스트 데이터셋으로 성능 평가

## 워크플로우
1. 데이터 다운로드 및 전처리
2. 데이터셋 로드 및 분할
3. 텍스트 벡터화 (정수 시퀀스 변환)
4. 모델 구축 (임베딩 + 양방향 LSTM)
5. 모델 훈련 및 평가

## 임베딩 vs 원핫 인코딩
- **임베딩 레이어**: 학습 가능한 밀집 벡터 표현, 메모리 효율적
- **원핫 인코딩**: 희소 벡터 표현, 메모리 사용량 높음
- **성능**: 임베딩이 일반적으로 더 나은 성능과 효율성 제공


## 1. 라이브러리 임포트

필요한 라이브러리들을 임포트합니다.


In [ ]:
import requests
import subprocess
import re
import string
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import os, pathlib, shutil, random
import keras
from keras import models, layers

print("TensorFlow 버전:", tf.__version__)
print("Keras 버전:", keras.__version__)


## 2. 데이터 다운로드 및 전처리 함수

IMDB 데이터셋을 다운로드하고 전처리하는 함수들을 정의합니다.


In [ ]:
def download():
    """IMDB 데이터셋을 다운로드하는 함수"""
    url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
    file_name = "aclImdb_v1.tar.gz"

    print("IMDB 데이터셋 다운로드 시작...")
    response = requests.get(url, stream=True)  # 스트리밍 방식으로 다운로드
    with open(file_name, "wb") as file:
        for chunk in response.iter_content(chunk_size=8192):  # 8KB씩 다운로드
            file.write(chunk)

    print("Download complete!")

def release():
    """압축을 해제하는 함수"""
    print("압축 해제 시작...")
    subprocess.run(["tar", "-xvzf", "aclImdb_v1.tar.gz"], shell=True)
    # tar.gz => linux에서는 파일을 여러개를 한번에 압축을 못함 
    # tar라는 형식으로 압축할 모든 파일을 하나로 묶어서 패키지로 만든다음에 압축을 한다.  
    # tar, gz가동 그래서 압축풀고 다시 패키지도 풀어야 한다. 
    # tar -xvzf 파일명 형태임         
    print("압축풀기 완료")

def labeling(): 
    """Train 데이터를 Train과 Validation으로 분할하는 함수"""
    print("데이터 라벨링 및 분할 시작...")
    base_dir = pathlib.Path("aclImdb") 
    val_dir = base_dir/"val"   # pathlib 객체에 / "디렉토리" => 결과가 문자열이 아니다 
    train_dir = base_dir/"train"

    # validation 디렉토리 생성 및 데이터 분할
    for category in ("neg", "pos"):
        os.makedirs(val_dir/category, exist_ok=True)  # 디렉토리를 만들고 
        files = os.listdir(train_dir/category)  # 해당 카테고리의 파일 목록을 모두 가져온다 
        random.Random(1337).shuffle(files)  # 파일을 랜덤하게 섞어서 복사하려고 파일 목록을 모두 섞는다 
        num_val_samples = int(0.2 * len(files))  # 20%를 validation으로 사용
        val_files = files[-num_val_samples:]  # 20%만 val폴더로 이동한다 
        for fname in val_files:
            shutil.move(train_dir/category/fname, val_dir/category/fname)
    
    print("데이터 라벨링 및 분할 완료")

# 주석 처리: 이미 데이터가 있다면 다시 다운로드할 필요 없음
# download()  # 파일 다운받기 = 용량이 너무 커서 8192만큼씩 잘라서 저장하는 코드임 
# release()   # 압축 해제
# labeling()  # 데이터 분할


## 3. 데이터셋 로드

Keras의 `text_dataset_from_directory`를 사용하여 디렉토리 구조로부터 데이터셋을 로드합니다.

- **0**: 부정 리뷰 (neg)
- **1**: 긍정 리뷰 (pos)


In [ ]:
# 배치 크기 설정
batch_size = 32  # 한번에 읽어올 양 

# 데이터셋 로드
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",  # 디렉토리명 
    batch_size=batch_size
)

val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val",  # 디렉토리명 
    batch_size=batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test",  # 디렉토리명 
    batch_size=batch_size
)

print("데이터셋 로드 완료!")
print(f"훈련 데이터: {len(train_ds)} 배치")
print(f"검증 데이터: {len(val_ds)} 배치") 
print(f"테스트 데이터: {len(test_ds)} 배치")


### 3.1 데이터 구조 확인

로드된 데이터의 구조와 내용을 확인해봅니다.


In [ ]:
# 데이터셋은 알아서 inputs, targets을 반복해서 갖고 온다. 우리한테 필요한거는 inputs만이다
for inputs, targets in train_ds:  # 실제 읽어오는 데이터 확인 
    print("inputs.shape", inputs.shape)
    print("inputs.dtype", inputs.dtype)
    print("targets.shape", targets.shape)
    print("targets.dtype", targets.dtype)
    print("\n=== 샘플 데이터 ===")
    print("inputs 샘플 (처음 3개):")
    for i, text in enumerate(inputs[:3]):
        print(f"  {i+1}: {text.numpy().decode('utf-8')[:100]}...")  # 처음 100자만 출력
    print(f"\ntargets 샘플 (처음 3개): {targets[:3]}")
    break  # 하나만 출력해보자 

print("\n=== 라벨 정보 ===")
print("0: 부정 리뷰 (neg)")  
print("1: 긍정 리뷰 (pos)")
print("폴더명을 정렬해서 0,1,2 이런식으로 라벨링을 한다 (neg -> 0, pos -> 1)")


## 4. 텍스트 벡터화

텍스트 데이터를 모델이 처리할 수 있는 정수 시퀀스로 변환합니다.

### 하이퍼파라미터 설정
- **max_length**: 한 리뷰에서 사용하는 최대 단어 수 (600)
- **max_tokens**: 자주 사용하는 단어 개수 (20000) - 어휘 사전 크기


In [ ]:
# 시퀀스 관련 하이퍼파라미터 설정
max_length = 600   # 한 평론에서 사용하는 단어는 최대 길이를 600개라고 보자  
max_tokens = 20000  # 자주 사용하는 단어 20000개만 쓰겠다 

# TextVectorization 레이어 생성
text_vectorization = TextVectorization( 
    max_tokens=max_tokens,
    output_mode="int",  # 임베딩 층을 사용하려면 반드시 int여야 한다
    output_sequence_length=max_length  
)

print(f"최대 시퀀스 길이: {max_length}")
print(f"어휘 사전 크기: {max_tokens}")
print("출력 모드: 정수 (int)")
print("TextVectorization 레이어 생성 완료!")


### 4.1 어휘사전 생성 및 벡터화 적용

훈련 데이터를 사용하여 어휘사전을 생성하고, 모든 데이터셋에 벡터화를 적용합니다.


In [ ]:
# 텍스트만 추출 (라벨 제거)
text_only_train_ds = train_ds.map(lambda x, y: x)

# 어휘사전 생성 (훈련 데이터 기반)
print("어휘사전 생성 중...")
text_vectorization.adapt(text_only_train_ds)  # 어휘사전을 만들어야 한다 
print("어휘사전 생성 완료!")

# 모든 데이터셋에 벡터화 적용
print("데이터셋 벡터화 중...")
int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y), 
    num_parallel_calls=1  # 병렬 처리 (원본 코드에서는 1)
)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y), 
    num_parallel_calls=1
)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y), 
    num_parallel_calls=1
)

print("데이터셋 벡터화 완료!")


### 4.2 벡터화된 데이터 확인

벡터화 후 데이터의 내부 구조를 확인해봅니다.


In [ ]:
# 벡터화된 데이터 내부구조 확인
print("=== 벡터화된 데이터 내부구조 살펴보기 ===")
for item in int_train_ds:
    vectorized_texts, labels = item
    print(f"벡터화된 텍스트 형태: {vectorized_texts.shape}")
    print(f"벡터화된 텍스트 타입: {vectorized_texts.dtype}")
    print(f"라벨 형태: {labels.shape}")
    print(f"라벨 타입: {labels.dtype}")
    
    print("\n=== 샘플 벡터화 결과 ===")
    print("첫 번째 리뷰의 벡터화 결과 (처음 20개 토큰):")
    print(vectorized_texts[0][:20])
    print(f"해당 라벨: {labels[0]}")
    
    # 0이 패딩을 의미하는지 확인
    non_zero_count = tf.reduce_sum(tf.cast(vectorized_texts[0] != 0, tf.int32))
    print(f"0이 아닌 토큰 개수: {non_zero_count} (전체 {max_length}개 중)")
    break


## 5. 임베딩 레이어와 모델 구축

### 임베딩 레이어의 장점
- **메모리 효율성**: 원핫 인코딩 대비 메모리 사용량 크게 감소
- **학습 가능**: 단어 간의 의미적 관계를 학습하여 밀집 벡터 생성
- **성능 향상**: 희소 벡터 대신 밀집 벡터로 더 나은 학습 성과

### 원핫 인코딩의 문제점
- **메모리 낭비**: 어휘 크기만큼의 희소 벡터 (20000차원에서 1개만 1, 나머지는 0)
- **학습 속도**: 희소 행렬로 인한 연산 비효율성
- **의미 표현**: 단어 간의 의미적 관계를 표현하지 못함

### 모델 아키텍처
1. **입력**: 정수 시퀀스 (배치_크기, 시퀀스_길이)
2. **임베딩**: 정수를 밀집 벡터로 변환 (배치_크기, 시퀀스_길이, 임베딩_차원)
3. **양방향 LSTM**: 양방향으로 시퀀스 처리
4. **드롭아웃**: 과적합 방지
5. **출력**: 이진 분류를 위한 시그모이드 활성화


In [ ]:
# 모델 하이퍼파라미터
embedding_dim = 256  # 임베딩 차원 (원본 코드에서 출력 차원)

# 입력 레이어 정의
inputs = keras.Input(shape=(None,), dtype="int64")
print(f"입력 형태: {inputs.shape}")

# 임베딩 레이어
# input_dim: 어휘 사전 크기 (20000)
# output_dim: 임베딩 벡터 차원 (256)
embedded = layers.Embedding(
    input_dim=max_tokens, 
    output_dim=embedding_dim
)(inputs) 

print(f"임베딩 후 형태: {embedded.shape}")
print(f"임베딩 설정: 입력 차원 {max_tokens} -> 출력 차원 {embedding_dim}")
print("임베딩 레이어를 통해 정수 시퀀스가 밀집 벡터로 변환됩니다.")

# 미리 학습된 임베딩층으로 바꿀 수도 있다는 주석
print("\n💡 참고: 미리 학습된 임베딩(Word2Vec, GloVe 등)으로 초기화 가능")


### 5.1 양방향 LSTM 및 출력 레이어 구성


In [ ]:
# 양방향 LSTM 레이어
x = layers.Bidirectional(layers.LSTM(32))(embedded) 
print("양방향 LSTM 추가 (32 유닛)")
print("- 순방향 LSTM: 문장의 앞에서 뒤로 정보 처리")
print("- 역방향 LSTM: 문장의 뒤에서 앞으로 정보 처리")
print("- 결과: 양방향 정보를 모두 고려한 더 풍부한 표현")

# 드롭아웃으로 과적합 방지
x = layers.Dropout(0.5)(x) 
print("\n드롭아웃 추가 (0.5)")
print("- 훈련 시 50% 뉴런을 무작위로 비활성화")
print("- 과적합 방지 및 일반화 성능 향상")

# 출력 레이어 (이진 분류)
outputs = layers.Dense(1, activation='sigmoid')(x)
print("\n출력 레이어 추가 (시그모이드 활성화)")
print("- 1개 뉴런: 이진 분류 (긍정/부정)")
print("- 시그모이드: 0~1 사이 확률값 출력")


### 5.2 모델 컴파일 및 구조 확인

모델을 생성하고 손실 함수, 옵티마이저, 메트릭을 설정합니다.


In [ ]:
# 모델 생성
model = keras.Model(inputs, outputs) 

# 모델 컴파일
model.compile(
    optimizer='rmsprop',           # RMSprop 옵티마이저
    loss='binary_crossentropy',    # 이진 분류용 손실 함수
    metrics=['accuracy']           # 정확도 메트릭
)

print("=== 모델 컴파일 완료 ===")
print("옵티마이저: RMSprop")
print("- 학습률을 적응적으로 조정하는 옵티마이저")
print("- RNN/LSTM에서 좋은 성능을 보임")
print("\n손실 함수: binary_crossentropy") 
print("- 이진 분류 문제에 적합한 손실 함수")
print("- 예측 확률과 실제 라벨 간의 교차 엔트로피 계산")
print("\n메트릭: accuracy")
print("- 정확히 예측한 샘플의 비율")

print("\n=== 모델 구조 요약 ===")
model.summary()


## 6. 모델 훈련

구축된 모델을 훈련 데이터로 학습시킵니다.

### 훈련 설정
- **에포크**: 10회
- **검증 데이터**: val 데이터셋 사용
- **verbose**: 1 (훈련 과정 상세 출력)


In [ ]:
# 모델 훈련 시작
print("=== 모델 훈련 시작 ===")
print("에포크: 10")
print("검증 데이터: val_ds")
print("임베딩 레이어를 사용하여 효율적인 학습 진행")
print()

# 훈련 실행
history = model.fit(
    int_train_ds, 
    validation_data=int_val_ds, 
    epochs=10,
    verbose=1  # 훈련 과정 출력
)

print("=== 모델 훈련 완료 ===")


## 7. 모델 평가

훈련이 완료된 모델을 테스트 데이터셋으로 평가하여 최종 성능을 확인합니다.


In [ ]:
# 테스트 데이터셋으로 모델 평가
print("=== 테스트 데이터셋 평가 ===")
test_loss, test_accuracy = model.evaluate(int_test_ds, verbose=1)

print(f"\n=== 최종 결과 ===")
print(f"테스트 손실: {test_loss:.4f}")
print(f"테스트 정확도: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# 훈련 히스토리 요약
if 'history' in locals():
    final_train_acc = history.history['accuracy'][-1]
    final_val_acc = history.history['val_accuracy'][-1]
    
    print(f"\n=== 훈련 과정 요약 ===")
    print(f"최종 훈련 정확도: {final_train_acc:.4f} ({final_train_acc*100:.2f}%)")
    print(f"최종 검증 정확도: {final_val_acc:.4f} ({final_val_acc*100:.2f}%)")
    print(f"테스트 정확도: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
    
    # 과적합 여부 확인
    if final_train_acc - test_accuracy > 0.1:
        print("\n⚠️  과적합 가능성이 있습니다.")
        print("- 드롭아웃 비율 증가 고려")
        print("- 조기 종료(Early Stopping) 적용 고려")
    else:
        print("\n✅ 적절한 일반화 성능을 보입니다.")

print(f"\n=== 간단 출력 (원본 스타일) ===")
print("테스트셋", [test_loss, test_accuracy])


## 8. 결론 및 개선사항

### 모델 특징
- **임베딩 레이어**: 정수 시퀀스를 밀집 벡터로 효율적 변환
- **양방향 LSTM**: 문맥을 양방향으로 고려하여 성능 향상
- **드롭아웃**: 과적합 방지를 위한 정규화

### 임베딩 레이어의 장점
1. **메모리 효율성**: 원핫 인코딩 대비 훨씬 적은 메모리 사용
2. **학습 가능**: 단어 간의 의미적 관계를 자동으로 학습
3. **성능 향상**: 밀집 벡터로 더 나은 표현력 제공
4. **전이 학습**: 사전 훈련된 임베딩 활용 가능

### 가능한 개선사항
1. **사전 훈련된 임베딩**: Word2Vec, GloVe, FastText 등 활용
2. **어텐션 메커니즘**: LSTM에 어텐션 추가로 중요한 부분에 집중
3. **Transformer 모델**: BERT, RoBERTa 등 최신 모델 활용
4. **하이퍼파라미터 튜닝**: 
   - 임베딩 차원 조정
   - LSTM 유닛 수 최적화
   - 드롭아웃 비율 조정
   - 학습률 스케줄링
5. **앙상블**: 여러 모델의 예측 결합
6. **데이터 증강**: 텍스트 증강 기법 적용

### 임베딩 vs 원핫 인코딩 비교

| 특성 | 임베딩 레이어 | 원핫 인코딩 |
|------|---------------|-------------|
| 메모리 사용량 | 적음 (차원×어휘크기) | 많음 (어휘크기×시퀀스길이) |
| 학습 속도 | 빠름 | 느림 |
| 의미 표현 | 우수 (단어 간 관계 학습) | 제한적 |
| 초기화 | 사전 훈련 모델 활용 가능 | 고정된 표현 |
| 권장 사용 | ✅ 대부분의 경우 | 특수한 실험 목적 |

### 실제 적용 시 고려사항
- **추론 속도**: 임베딩 레이어가 원핫 인코딩보다 빠름
- **모델 크기**: 임베딩 가중치 매트릭스 크기 고려
- **도메인 특화**: 특정 도메인 데이터로 임베딩 파인튜닝
- **다국어 지원**: 다국어 임베딩 모델 활용
